In [1]:
try:
    import eomaji
except ModuleNotFoundError:
    !pip install eomaji@git+https://github.com/DHI/EOMAJI-OpenEO-toolbox.git

# Prepare data for PyDMS and ET Flows

This notebook can be run on the Copernicus Dataspace Jupyterhub but running the following package installation cell first

**Note** You should select on of the kernels with GDAL installed, eg. "Geo science"

In [12]:
import os 
import openeo
from eomaji.utils import draw_utils, date_selector
from eomaji.utils.general_utils import dump_area_date_info, read_area_date_info
from eomaji.workflows import prepare_data_cubes

## 1. Select Area of Interest
Draw a polygon on the map, for the area you want to process

In [13]:
map, bboxs = draw_utils.draw_aoi()
map

Map(center=[40, 10], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

## 2. Select Date from Available Dates for Sentinel-3
Search for days where sentinel 3 images are available for the given AOI. 

**Note**: max_cloud_cover refers to the full tile’s coverage, not just your AOI, so results may vary.

In [4]:
# Define search parameters
start_date = "2023-05-01"
end_date = "2023-07-30"
bbox = bboxs[-1] # If you didn't draw a polygon, please insert a bbox here in the form of [minx, miny, maxx, maxy]
max_cloud_cover = 10  # Filter out high-cloud-coverage scenes

# Search for available Sentinel-3 imagery
date_selection = date_selector.get_available_dates(
    start_date=start_date,
    end_date=end_date,
    bbox=bbox,
    max_cloud_cover=max_cloud_cover
)

Dropdown(description='Pick a Date:', options=(('2023-05-12', datetime.date(2023, 5, 12)), ('2023-05-31', datet…

## 3. Connect to OpenEO Backend

In [14]:
connection = openeo.connect("https://openeo.dataspace.copernicus.eu")
connection.authenticate_oidc()

2025-06-23 12:52:16,432 [INFO] Found OIDC providers: ['CDSE']
2025-06-23 12:52:16,438 [INFO] No OIDC provider given, but only one available: 'CDSE'. Using that one.
2025-06-23 12:52:16,764 [INFO] Using default client_id 'sh-b1c3a958-52d4-40fe-a333-153595d1c71e' from OIDC provider 'CDSE' info.
2025-06-23 12:52:16,844 [INFO] Found refresh token: trying refresh token based authentication.
2025-06-23 12:52:16,847 [INFO] Doing 'refresh_token' token request 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token' with post data fields ['grant_type', 'client_id', 'refresh_token'] (client_id 'sh-b1c3a958-52d4-40fe-a333-153595d1c71e')
2025-06-23 12:52:17,390 [INFO] Obtained tokens: ['access_token', 'id_token', 'refresh_token']
2025-06-23 12:52:17,393 [INFO] Storing refresh token for issuer 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE' (client 'sh-b1c3a958-52d4-40fe-a333-153595d1c71e')


Authenticated using refresh token.


<Connection to 'https://openeo.dataspace.copernicus.eu/openeo/1.2/' with OidcBearerAuth>

## 4. Download Sentinel 2 and Sentinel 3 data for the specified AOI and date

In [ ]:
data_dir = "./data"
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

date = date_selection.value
#date, bbox = read_area_date_info(
#    dir=data_dir
#) # <-- USE THIS IF YOU WANT TO REUSE BBOX AND DATE FROM FILE

In [17]:
s2_path, s3_path, vza_path, worldcover_path, dem_s2_path, dem_s3_path, sentinel3_acq_time, = prepare_data_cubes.prepare_data_cubes(
    connection=connection,
    bbox=bbox,
    date=date,
    sentinel2_search_range = 3,
    out_dir = data_dir,
)

2025-06-23 12:52:30,441 [INFO] Cached Sentinel 2 data cube found. Skipping download.
/usr/local/lib/python3.12/dist-packages/openeo/rest/connection.py:1171: UserWarning: Property filtering with unsupported properties according to collection/STAC metadata: {'timeliness', 'orbitDirection'} (supported: dict_keys(['bands', 'eo:bands', 'raster:bands'])).
  return DataCube.load_collection(
2025-06-23 12:52:46,422 [INFO] Cached DEM data cube found. Skipping download.
2025-06-23 12:52:46,425 [INFO] Cached DEM cube found. Skipping download.
2025-06-23 12:52:46,501 [INFO] Cached Worldcover cube found. Skipping download.
2025-06-23 12:52:46,781 [INFO] Job j-2506231252314ad5ab9272dadc197369 status: created
2025-06-23 12:52:57,034 [INFO] Job j-2506231252314ad5ab9272dadc197369 status: created
2025-06-23 12:53:07,230 [INFO] Job j-2506231252314ad5ab9272dadc197369 status: created
2025-06-23 12:53:17,428 [INFO] Job j-2506231252314ad5ab9272dadc197369 status: running
2025-06-23 12:53:27,846 [INFO] Job j-2

### Store AOI and Date for use in other notebooks    

In [18]:
dump_area_date_info(
    date = date, 
    bbox = bbox, 
    out_dir = data_dir
)